In [1]:
import utils
import DataProcess
from pprint import pprint

In [2]:
start_timesatmp = utils._convert_to_timestamp_ms('2024-12-23 00:00:00')
log_data = DataProcess.TradingLog(symbol='ABC',
                                  start_timestamp=start_timesatmp,
                                  entry_price = 65000,
                                  test_mode = False,
                                  position=1,
                                  quantity=1_000,
                                  adj_rate=0.001,
                                  adj_interval='3m',
                                  leverage=10,
                                  use_scale_stop=True,
                                  stop_rate=0.025,
                                  trade_scenario='case_1')

AttributeError: 'TradingLog' object has no attribute 'symbol'

In [46]:
data_container = utils.DataContainer()
data_container.set_data(log_data.symbol, log_data)

In [50]:
data_container.get_data(log_data.symbol).update_trade_data(current_price=67000, current_timestamp=start_timesatmp)

0.7030000000000001
180000
0.001
0.015


In [53]:
data_container.get_data(log_data.symbol).to_list()

['ABC',
 1734879600000,
 65000,
 1,
 1000,
 10,
 'case_1',
 False,
 0.025,
 0.05,
 0.015,
 True,
 0.001,
 '3m',
 113096.00000000001,
 68152.4,
 True,
 1735006219532,
 67000,
 65000,
 67000,
 6500000.0,
 6700000.0,
 200000.0,
 134000.0,
 65066.0,
 32500.0,
 33500.0,
 1934000.0]

In [3]:
import BinanceTradeClient
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [4]:
obj_futures = BinanceTradeClient.FuturesOrder()

In [5]:
asyncio.run(obj_futures.set_leverage(symbol='XRPUSDT', leverage=5))
asyncio.run(obj_futures.set_margin_type(symbol='XRPUSDT', margin_type='isolated'))
data = asyncio.run(obj_futures.submit_order(symbol='XRPUSDT', side='buy', order_type='market', quantity=2.4))

In [6]:
print(data)

{'orderId': 78182197291, 'symbol': 'XRPUSDT', 'status': 'NEW', 'clientOrderId': 'IVvdjAcOS12gJZezQCC4by', 'price': '0.0000', 'avgPrice': '0.00', 'origQty': '2.4', 'executedQty': '0.0', 'cumQty': '0.0', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.0000', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1735017118771}


In [8]:
dummy_data = asyncio.run(obj_futures.fetch_account_balance())

In [12]:
for i in dummy_data['positions']:
    if i.get('symbol') == 'XRPUSDT':
        pprint(i)

{'askNotional': '0',
 'bidNotional': '0',
 'breakEvenPrice': '2.26143015',
 'entryPrice': '2.2603',
 'initialMargin': '1.08350400',
 'isolated': True,
 'isolatedWallet': '1.08765636',
 'leverage': '5',
 'maintMargin': '0.02708760',
 'maxNotional': '20000000',
 'notional': '5.41752000',
 'openOrderInitialMargin': '0',
 'positionAmt': '2.4',
 'positionInitialMargin': '1.08350400',
 'positionSide': 'BOTH',
 'symbol': 'XRPUSDT',
 'unrealizedProfit': '-0.00720000',
 'updateTime': 1735017118771}


In [13]:
asyncio.run(obj_futures.fetch_trade_history(symbol='XRPUSDT', limit=1))

[{'symbol': 'XRPUSDT',
  'id': 1916252682,
  'orderId': 78182197291,
  'side': 'BUY',
  'price': '2.2603',
  'qty': '2.4',
  'realizedPnl': '0',
  'quoteQty': '5.42472',
  'commission': '0.00271236',
  'commissionAsset': 'USDT',
  'time': 1735017118771,
  'positionSide': 'BOTH',
  'buyer': True,
  'maker': False}]

In [20]:
trade_message = {}
if not trade_message:
    print(True)

True


In [18]:
leverage

2

In [24]:
a = ['a']
b = []

b.append(a.pop())

In [25]:
a

[]

In [27]:
dummy = {"a":[]}

In [29]:
if not dummy.get('a'):
    print(True)

True


In [30]:
if 'a' in dummy.keys():
    print(True)

True
